In [76]:

import tkinter as tk
from tkinter import ttk
import openpyxl
from pandas import read_excel
import numpy as np
from numpy import nan
import tkinter.filedialog
import tkinter.messagebox
import pandas as pd  
from tkinter import messagebox           


def error_tab(frame1):
    dic = {"Issue": 3, 
            "Action Taken": 4, 
            "Frequency": 5
            }
    
    global combo, sops, labels, entries, comboboxlist, buttons, tk_list
    buttons = []
    labels = []
    entries = []
    sops = []
    comboboxlist = []
    tk_list = []
    
    errors = getErrorList()
    errors.append("Add new error")


    # Clear existing widgets
    for widget in frame1.winfo_children():
        widget.destroy()

    # Create combo box
    global selected_error_var
    selected_error_var = tk.StringVar()
    combo = ttk.Combobox(frame1, values=errors, state="readonly", width=40, textvariable=selected_error_var)
    combo.grid(row=2, column=0, sticky='W')
    # Optionally, remove or adjust this if you want a different default:
    combo.current(len(errors)-1)

    # combo = ttk.Combobox(frame1, values=errors, state="readonly", width=40)
    # combo.grid(row=2, column=0, sticky='W')
    # combo.current(len(errors)-1)
    
    def display_query_fields():
        # display all things to query
        for key, value in dic.items():
            label = tk.Label(frame1, text=key)
            label.grid(row=value, column=0, sticky='W')
            labels.append(label)
            provided = tk.StringVar()
            entry = tk.Entry(frame1, width=35, textvariable=provided)
            entry.grid(row=value, column=1, sticky='WE')
            entries.append(entry)
        button = tk.Button(frame1, text="Save", width=5, height=1, command=SaveNewError)
        button.place(relx=0.5, rely=0.5, anchor="se")
        buttons.append(button)

    # def Display_Delete_page():
    #     label = tk.Label(frame1, text="Select Error to Delete")
    #     label.grid(row=3, column=0, sticky='W')
    #     labels.append(label)

    #     ErrorComboBox = ttk.Combobox(frame1, values=errors, state="readonly")
    #     ErrorComboBox.grid(row=3, column=2, sticky='W')
    #     ErrorComboBox.current(len(errors)-1)
    #     comboboxlist.append(ErrorComboBox)    
        
    def display_Current(ComboBoxValue):
        # display all things to query
        label = tk.Label(frame1, text=ComboBoxValue)
        label.grid(row=3, column=0, sticky='W')  
        labels.append(label)
        
        label1 = tk.Label(frame1, text="Increment or Decrement Frequency")
        label1.grid(row=3, column=1, sticky='W')  
        labels.append(label1)

        # Create a combo box, values= getActionList(ComboBoxValue)
        actions = getActionList(ComboBoxValue)
        actions.append("Delete Action")
        actions.append("Add Action")        

        label = tk.Label(frame1, text="Action Taken")
        label.grid(row=4, column=0, sticky='W')
        labels.append(label)
        ActionComboBox = ttk.Combobox(frame1, values=actions, state="readonly", width=40)
        ActionComboBox.grid(row=4, column=1, sticky='W')
        ActionComboBox.current(len(actions)-1)
        comboboxlist.append(ActionComboBox) # append to list to destory everytime we reset
        
        
        
        
        
        
        def update_Action(event):
            global tk_list  # Declare tk_list as a global variable again
            current_Action = ActionComboBox.current()
            ActionValue = actions[current_Action]
            for item in tk_list:
                item.destroy()
            tk_list = []
            match ActionValue: 
                case "Add Action":
                    # Clear any previous widgets before adding new ones
                    global label, frequency_label, plus_button, minus_button

                    # print("Okay this is the Default value")

                    # Label for entering new action
                    labelf = tk.Label(frame1, text="Enter Action:")
                    labelf.grid(row=5, column=0, sticky='W')
                    tk_list.append(labelf)

                    # Entry field for new action
                    providedf = tk.StringVar()
                    entryq = tk.Entry(frame1, width=35, textvariable=providedf)
                    entryq.grid(row=5, column=1, sticky='W')
                    tk_list.append(entryq)

                    # Create the "Save Action" button
                    save_action_button = tk.Button(
                        frame1, text="Save Action", width=20, 
                        command=lambda: save_action(ComboBoxValue, providedf.get(), 1)  # Corrected variable reference
                    )
                    save_action_button.grid(row=6, column=1, pady=10, sticky="W")  # Align with Entry
                    tk_list.append(save_action_button)
                    
                # case "Delete Action":
                #     global label, frequency_label, plus_button, minus_button
                    
                #     # Label for entering new action
                #     labelf = tk.Label(frame1, text="Enter Action:")
                #     labelf.grid(row=5, column=0, sticky='W')
                #     tk_list.append(labelf)
                
                #     # Create a new combobox with values= action list
                #     actions = getActionList(ComboBoxValue)
                #     DeleteActionComboBox = ttk.Combobox(frame1, values=actions, state="readonly", width=40)
                #     DeleteActionComboBox.grid(row=5, column=1, sticky='W')
                #     tk_list.append(DeleteActionComboBox)

                #     # Bind the combobox selection to a function
                #     def delete_action_confirm(event):
                        
                #         bruhaction = getActionList(ComboBoxValue)
                #         deleteAction = ActionComboBox.current()
                #         ActionValue = bruhaction[current_Action]
                #         deleteAction(ComboBoxValue,ActionValue)
                        
                #     # def deleteAction(ErrorString, ActionToDel):

                #     DeleteActionComboBox.bind("<<ComboboxSelected>>", delete_action_confirm)
                case "Delete Action":
                    # Label for selecting action to delete
                    delete_label = tk.Label(frame1, text="Select Action to Delete:")
                    delete_label.grid(row=5, column=0, sticky='W')
                    tk_list.append(delete_label)

                    # Get the list of actions for the selected error
                    newactionlist = getActionList(ComboBoxValue)

                    # Create a combobox with available actions
                    delete_action_combobox = ttk.Combobox(frame1, values=newactionlist, state="readonly", width=40)
                    delete_action_combobox.grid(row=5, column=1, sticky='W')
                    tk_list.append(delete_action_combobox)

                    # Delete Button
                    def confirm_delete():
                        selected_action = delete_action_combobox.get()
                        if not selected_action:
                            messagebox.showerror("Error", "Please select an action to delete.")
                            return

                        confirmation = messagebox.askyesno("Confirm Deletion", f"Are you sure you want to delete '{selected_action}'?")
                        if confirmation:
                            deleteAction(ComboBoxValue, selected_action)
                            messagebox.showinfo("Success", f"Action '{selected_action}' deleted successfully.")
                            delete_action_combobox['values'] = getActionList(ComboBoxValue)  # Refresh the combobox

                    delete_button = tk.Button(frame1, text="Delete Action", command=confirm_delete, width=15)
                    delete_button.grid(row=6, column=1, sticky='W', pady=10)
                    tk_list.append(delete_button)

                
                case _:
                    # Label for frequency
                    labeld = tk.Label(frame1, text="Frequency")
                    labeld.grid(row=5, column=0, sticky='W')
                    tk_list.append(labeld)

                    # Frequency Label
                    frequency_label = tk.Label(
                        frame1,
                        text=getActionFrequency(ComboBoxValue, ActionValue),
                        font=('Helvetica', 20),
                        anchor='w'
                    )
                    frequency_label.grid(row=5, column=1, sticky='W', padx=(5, 0))  # Move it closer
                    tk_list.append(frequency_label)

                    # Function to update the label
                    def update_label():
                        frequency_label.config(text=getActionFrequency(ComboBoxValue, ActionValue))

                    # # Configure columns to prevent unnecessary stretching
                    # # Configure grid column weights to prevent unwanted stretching
                    # frame1.grid_columnconfigure(0, weight=0)  # Column 0 won't expand
                    # frame1.grid_columnconfigure(1, weight=1)  # Column 1 (buttons) will expand as needed
                    # frame1.grid_columnconfigure(2, weight=0)  # Column 2 won't expand

                    # # If you have specific row height requirements, you can adjust them
                    # frame1.grid_rowconfigure(4, weight=0)  # Prevent row 5 (where buttons are) from stretching

                    # Button frame placement
                    button_frame = tk.Frame(frame1)
                    button_frame.grid(row=6, column=1, sticky='W', padx=10)  # Adjust column for proper alignment
                    tk_list.append(button_frame)  # Append the frame to tk_list

                    # Plus Button
                    plus_button = tk.Button(
                        button_frame, text="+", font=('Helvetica', 12),
                        width=5, height=2,  # Bigger buttons
                        command=lambda: [IncrementActionCount(ComboBoxValue, ActionValue), update_label()]
                    )
                    plus_button.pack(side="left", padx=5)  # Keep buttons close
                    tk_list.append(plus_button)  # Append the plus button to tk_list


                    # Minus Button
                    minus_button = tk.Button(
                        button_frame, text="-", font=('Helvetica', 12),
                        width=5, height=2,  # Bigger buttons
                        command=lambda: [DecrementActionCount(ComboBoxValue, ActionValue), update_label()]
                    )
                    minus_button.pack(side="left", padx=5)  # Keep buttons close
                    tk_list.append(minus_button)  # Append the minus button to tk_list




        ActionComboBox.bind("<<ComboboxSelected>>", update_Action)
    
    # # Whenever there is a change in combo            
    # def error_page(event):
    #     global labels, entries, comboboxlist, buttons
    #     # Remove existing labels and entries
    #     for label in labels:
    #         label.destroy()
    #     for entry in entries:
    #         entry.destroy()
    #     for item in comboboxlist:
    #         item.destroy()
    #     for item in buttons:
    #         item.destroy()
    #     labels = []
    #     entries = []
    #     comboboxlist = []
    #     buttons = []
        
    #     current = combo.current()
    #     value = errors[current]
    #     print(f"bruh: {value}")
    #     match value: 
    #         case "Add new error":
    #             print("Only ADD")
    #             display_query_fields()
    #         case _:
    #             print("REST")
    #             display_Current(value)
    #     # if value == "Add new error":
    #     #     print("Only ADD")
    #     #     display_query_fields()
    #     # else:
    #     #     print("REST")
    #     #     display_Current(value)


    # combo.bind("<<ComboboxSelected>>", error_page)        
    
    def error_page(event):
        global labels, entries, comboboxlist, buttons, tk_list
        
        # Remove existing widgets
        for label in labels:
            label.destroy()
        for entry in entries:
            entry.destroy()
        for item in comboboxlist:
            item.destroy()
        for item in buttons:
            item.destroy()
        for item in tk_list:
            item.destroy()
        labels = []
        entries = []
        comboboxlist = []
        buttons = []
        tk_list = []

        # Get the selected value directly from the combo box
        value = selected_error_var.get()
        print(f"Selected error: {value}")  # Debugging output

        # Check if the value is "Add new error" or something else
        if value == "Add new error":
            print("Only ADD")
            display_query_fields()  # Function to add new error
        else:
            print("REST")
            display_Current(value)  # Function to display current error and actions

    # Bind combo box selection to error_page function
    combo.bind("<<ComboboxSelected>>", error_page)



In [84]:
if __name__ == "__main__":
    main()

Found nan at row: 414
Found nan at row: 362
Found nan at row: 385
Found nan at row: 404
Found nan at row: 412
Found nan at row: 404
Found nan at row: 412
Found nan at row: 414
Found nan at row: 412


In [77]:
def Action_Order_Tab(frame2):
    global combo, sops, labels, entries, comboboxlist, buttons
    buttons = []
    labels = []
    entries = []
    sops = []
    comboboxlist = []
    
    errors = getErrorList()
    # errors.append("Add new error")

    # Clear existing widgets
    for widget in frame2.winfo_children():
        widget.destroy()


    # Create combo box
    # combo = ttk.Combobox(frame2, values=errors, state="readonly", width=40)
    # combo.grid(row=2, column=0, sticky='W')
    # combo.current(len(errors)-1)
    
    # Create combo box for error selection
    selected_error_var = tk.StringVar()
    error_combo = ttk.Combobox(frame2, values=errors, state="readonly", width=40, textvariable=selected_error_var)
    error_combo.grid(row=0, column=0, sticky='W')
    error_combo.current(len(errors) - 1)
    
    def update_action_order(event):
        # Clear previous labels and comboboxes
        for label in labels:
            label.destroy()
        for combo in comboboxlist:
            combo.destroy()
        labels.clear()
        comboboxlist.clear()

        selected_error = selected_error_var.get()
        if selected_error == "Add new error":
            return
        
        actions = getActionList(selected_error)
        num_actions = len(actions)

        for i, action in enumerate(actions):
            label = tk.Label(frame2, text=f"{action} Order:")
            label.grid(row=i+1, column=0, sticky='W')
            labels.append(label)

            order_var = tk.StringVar()
            action_combo = ttk.Combobox(frame2, values=list(range(1, num_actions+1)), state="readonly", textvariable=order_var, width=5)
            action_combo.grid(row=i+1, column=1, sticky='W')
            comboboxlist.append(action_combo)

    # Bind error selection to update function
    error_combo.bind("<<ComboboxSelected>>", update_action_order)
    

In [86]:
import tkinter as tk
from tkinter import ttk

def Action_Order_Tab(frame2):
    global combo, sops, labels, entries, comboboxlist, buttons
    buttons = []
    labels = []
    entries = []
    sops = []
    comboboxlist = []
    
    errors = getErrorList()

    # Clear existing widgets
    for widget in frame2.winfo_children():
        widget.destroy()

    # Create a canvas and a frame within it
    canvas = tk.Canvas(frame2, width=800, height=600)
    scrollbar = tk.Scrollbar(frame2, orient=tk.VERTICAL, command=canvas.yview)
    
    scroll_frame = tk.Frame(canvas)

    # Attach scrollbar and frame
    scroll_window = canvas.create_window((0, 0), window=scroll_frame, anchor="nw")

    canvas.configure(yscrollcommand=scrollbar.set)

    # Pack everything
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Function to update scroll region
    def update_scroll_region(event=None):
        canvas.configure(scrollregion=canvas.bbox("all"))

    scroll_frame.bind("<Configure>", update_scroll_region)

    # Enable mousewheel scrolling with scroll limits
    def on_mousewheel(event):
        current_scroll = canvas.yview()
        if (event.delta > 0 and current_scroll[0] <= 0.0) or (event.delta < 0 and current_scroll[1] >= 1.0):
            return  # Prevent scrolling beyond limits
        canvas.yview_scroll(-1 * (event.delta // 120), "units")

    canvas.bind_all("<MouseWheel>", on_mousewheel)  # For Windows
    canvas.bind_all("<Button-4>", lambda e: canvas.yview_scroll(-1, "units"))  # For Linux (scroll up)
    canvas.bind_all("<Button-5>", lambda e: canvas.yview_scroll(1, "units"))   # For Linux (scroll down)

    # Create combo box for error selection
    selected_error_var = tk.StringVar()
    error_combo = ttk.Combobox(scroll_frame, values=errors, state="readonly", width=40, textvariable=selected_error_var)
    error_combo.grid(row=0, column=0, sticky='W')
    error_combo.current(len(errors) - 1)
    
    def update_action_order(event):
        # Clear previous labels and comboboxes
        for label in labels:
            label.destroy()
        for combo in comboboxlist:
            combo.destroy()
        labels.clear()
        comboboxlist.clear()

        selected_error = selected_error_var.get()
        if selected_error == "Add new error":
            return
        
        actions = getActionList(selected_error)
        num_actions = len(actions)

        for i, action in enumerate(actions):
            label = tk.Label(scroll_frame, text=f"{action} Order:")
            label.grid(row=i+1, column=0, sticky='W')
            labels.append(label)

            order_var = tk.StringVar()
            action_combo = ttk.Combobox(scroll_frame, values=list(range(1, num_actions+1)), state="readonly", textvariable=order_var, width=5)
            action_combo.grid(row=i+1, column=1, sticky='W')
            comboboxlist.append(action_combo)

        update_scroll_region()

    # Bind error selection to update function
    error_combo.bind("<<ComboboxSelected>>", update_action_order)



In [87]:
if __name__ == "__main__":
    main()

Found nan at row: 362
Found nan at row: 299
Found nan at row: 323
Selected error: Disk slanted
REST
Found nan at row: 362
Found nan at row: 176
Found nan at row: 248
Found nan at row: 248


# Global Functions

In [78]:
def get_user_input():
    user_inputs = []
    for i, entry in enumerate(entries):
        user_input = entry.get()  # or entries[i].get()
        user_inputs.append(user_input)
    return user_inputs

def SaveNewError():
    user_inputs = get_user_input()
    try:
        result = addError(user_inputs[0],user_inputs[1],float(user_inputs[2]))
        if result is False:
            raise Exception("Error already exists")
        
        # errors = getErrorList()
        # errors.append("Add new error")
        # combo['values'] = errors
        # selected_error_var.set("Add new error")
        messagebox.showinfo("Success", f"{user_inputs[0]} successfully added.")
        error_tab(frame1) # Go back ErrorTab

    except Exception as e:
        print(e)
        if str(e) == "Error already exists":
            messagebox.showerror("Error", "Error already exists.")
        else:
            messagebox.showerror("Error", "Please enter a number for frequency.")
    # Get the ErrorString
    # Call addError(ErrorString):
    
            # def addNewAction(ErrorString, newActionString,Frequency):

def save_action(error_name,givenActionName,freq):
    """Retrieves user input and saves a new action for the selected error."""

    if not givenActionName.strip():
        messagebox.showerror("Error", "Action name cannot be empty.")
        return

    result = addNewAction(error_name, givenActionName, freq)  # Default frequency is 1

    if result:
        messagebox.showinfo("Success", "Action added successfully!")
        


# Main GUI

In [80]:
def create_gui():
    global frame1,frame2,frame3
    root = tk.Tk()
    root.geometry('600x450')
    root.title('Self-Update Platform')

    notebook = ttk.Notebook(root)
    notebook.pack(pady=10, expand=True)

    frame1 = ttk.Frame(notebook, width=600, height=400)
    frame2 = ttk.Frame(notebook, width=600, height=400)
    frame3 = ttk.Frame(notebook, width=600, height=400)

    frame1.pack(fill='both', expand=True)
    frame2.pack(fill='both', expand=True)
    frame3.pack(fill='both', expand=True)

    notebook.add(frame1, text='Error Entry')
    notebook.add(frame2, text='Action Order')
    notebook.add(frame3, text='Work In Progress')

    return root, frame1, frame2, frame3
def main():
    root, frame1, frame2, frame3 = create_gui()
    error_tab(frame1)
    Action_Order_Tab(frame2)
    # loop_tab(frame3) # uncoment
    root.mainloop()

if __name__ == "__main__":
    main()

Selected error: Disk slanted
REST
Found nan at row: 362
Found nan at row: 362
Start row: 353
"Verify load unload dfcd with disk" found at index: 356
8.0
Found nan at row: 412
Found nan at row: 343
Found nan at row: 156


# Read CSV

In [ ]:
import pandas as pd
import csv
file_path = r"C:\Users\768604\Desktop\maint_app\merged_output\COLLATED EVERYTHING.csv"
df = pd.read_csv(file_path)
df.head()

# API Backend

In [79]:
import pandas as pd
import csv
 
def getErrorDict():
    df = pd.read_csv(file_path)
    ErrorDict={}
    for index, row in enumerate(df.values):
        if str(row[0]) != "nan":
            ErrorDict[row[0]]=index
    # print(ErrorDict)
    return ErrorDict

def getErrorList():
    ErrorDict=getErrorDict()
    return list(ErrorDict.keys())

# Get the row index of Error
def getCurrentRow(ErrorString):
    ErrorDict = getErrorDict()
    try:
        return ErrorDict[ErrorString]
    except KeyError:
        print("No Such Issue Found!")
        return None  # or raise a custom exception

def getActionList(ErrorString):
    ActionList=[]
    df = pd.read_csv(file_path)
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return

    lastrow = len(df)  # Default to the last row if no 'nan' is found

    for index, row in enumerate(df.values[start_row:], start=start_row):
        # print(index)
        # print(str(row[1]))
        if str(row[1]) == "nan":
            # Add your action here
            lastrow = index
            print(f"Found nan at row: {lastrow}")
            break
        else:
            ActionList.append(str(row[1]))
        
    # print(ActionList)    
    return ActionList

def getLastRowIndex():
    df = pd.read_csv(file_path)
    last_row = df.iloc[-1]  # get the last row
    return last_row.name

def getActionFrequency(ErrorString, ActionString):
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return
    
    print(f"Start row: {start_row}")
    df = pd.read_csv(file_path)
    # Identify the last row using pandas
    RowToChange=None
    for index, row in enumerate(df.values[start_row:], start=start_row):
        # print(f"Index: {index}, Value of row[1]: {row[1]}")
        if str(row[1]) == ActionString:
            RowToChange=index
            print(f'"{ActionString}" found at index: {RowToChange}')
            # row[2]=row[2]+1
            print(row[2])
            # print(row[2]) 
            return row[2]

        elif str(row[1]) == "nan": #idk need to test later
            print("No Action Found!")
            return

def addError(ErrorString, ActionTaken, Frequency):
    try:
        if ErrorString in getErrorList():
            return False
        
        df = pd.read_csv(file_path)
        lastrow = getLastRowIndex()
        print(lastrow)

        emptyrow=["","",""]
        new_row = [ErrorString, ActionTaken, Frequency]  # create a new row
        
        with open(file_path, 'r') as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        rows.insert(lastrow+2, emptyrow)
        rows.insert(lastrow+3, new_row)
        
        with open(file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(rows)
        print("New action added successfully.")
        return True
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

def addNewAction(ErrorString, newActionString,Frequency):
    df = pd.read_csv(file_path)
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return

    lastrow = len(df)  # Default to the last row if no 'nan' is found

    for index, row in enumerate(df.values[start_row:], start=start_row):
        print(index)
        print(str(row[1]))
        if str(row[1]) == "nan":
            # Add your action here
            lastrow = index
            print(f"Found nan at row: {lastrow}")
            break
    else:
        print("No nan value found!")
        
    # print("hello")
    new_row = [None, newActionString, Frequency]  # create a new row
    
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)
    
    rows.insert(lastrow+1, new_row)
    
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(rows)
    print("New action added successfully.")
    return True

# addNewAction("Load Head Issue","HELLLOOOOOO",12)

def IncrementActionCount(ErrorString,ActionString):
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return
    
    print(f"Start row: {start_row}")
    df = pd.read_csv(file_path)
    # Identify the last row using pandas
    RowToChange=None
    for index, row in enumerate(df.values[start_row:], start=start_row):
        # print(f"Index: {index}, Value of row[1]: {row[1]}")
        if str(row[1]) == ActionString:
            RowToChange=index
            print(f'"{ActionString}" found at index: {RowToChange}')
            # row[2]=row[2]+1
            df.at[RowToChange, df.columns[2]] += 1     
            # print(row[2]) 
            break

        elif str(row[1]) == "nan": #idk need to test later
            return
        
    df.to_csv(file_path, index=False)

def DecrementActionCount(ErrorString, ActionString):
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return
    
    print(f"Start row: {start_row}")
    df = pd.read_csv(file_path)
    # Identify the last row using pandas
    RowToChange=None
    for index, row in enumerate(df.values[start_row:], start=start_row):
        # print(f"Index: {index}, Value of row[1]: {row[1]}")
        if str(row[1]) == ActionString:
            RowToChange=index
            print(f'"{ActionString}" found at index: {RowToChange}')
            # Decrement the count, but not below 0
            df.at[RowToChange, df.columns[2]] = max(0, df.at[RowToChange, df.columns[2]] - 1)     
            break

        elif str(row[1]) == "nan": 
            return
        
    df.to_csv(file_path, index=False)

def deleteError(ErrorString):
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return

    print(f"Start row: {start_row}")
    lastrow = len(df)  # Default to the last row if no 'nan' is found
    
    df = pd.read_csv(file_path)

    # Identify the last row using pandas
    for index, row in enumerate(df.values[start_row:], start=start_row):
        print(f"Index: {index}, Value of row[1]: {row[1]}")
        if str(row[1]) == "nan":
            lastrow = index
            print(f"Last row found at index: {lastrow}")
            break

    if lastrow is None:
        print("No valid last row found. Exiting.")
        return

    # Read the CSV using csv.reader
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)

    # Adjust for header if applicable
    header_offset = 1 if reader and isinstance(reader, list) and reader[0] else 0
    start_row += header_offset
    lastrow += header_offset

    # Delete rows from start_row to lastrow (inclusive)
    print(f"Deleting rows from {start_row} to {lastrow}")
    del rows[start_row:lastrow + 1]

    # Write updated data
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    print("Error deleted successfully.")

def deleteAction(ErrorString, ActionToDel):
    
    start_row = getCurrentRow(ErrorString)
    if start_row is None:
        print("Issue not found")
        return

    print(f"Start row: {start_row}")
    df = pd.read_csv(file_path)
    # Identify the last row using pandas
    RowToDel=None
    for index, row in enumerate(df.values[start_row:], start=start_row):
        # print(f"Index: {index}, Value of row[1]: {row[1]}")
        if str(row[1]) == ActionToDel:
            RowToDel=index
            print(f'"{ActionToDel}" found at index: {RowToDel}')
            break

        elif str(row[1]) == "nan": #idk need to test later
            return
    # print("HELLOE WORLD")
    if RowToDel==None:
        print(f'"{ActionToDel}" Not Found in "{ErrorString}"!')
        return        
    # Read the CSV using csv.reader
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)

    # Adjust for header if applicable
    header_offset = 1 if reader and isinstance(reader, list) and reader[0] else 0
    start_row += header_offset
    
    print(f"Deleting rows from {RowToDel}")
    if RowToDel==start_row:
        print("Please delete Entire Error Record instead!")
        return
    else:
        del rows[RowToDel+1]
        
    # Write updated data
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    print("Error deleted successfully.")


import pandas as pd
file_path = r"C:\Users\768604\Desktop\maint_app\merged_output\COLLATED EVERYTHING.csv"             
df = pd.read_csv(file_path)

# ErrorString="Amp low"  
# ErrorString="spindle fault"
ErrorString="Dover error"


# NewActionString="bruh" 
# ActiontoDelete="Adjust Left Preamp Air" 
# ActiontoDelete="Put back original Dover"   
# ActiontoDelete="Reset VXI"
# brrow=getCurrentRow(ErrorString)
# print(brrow)
ActiontoChange="Adjust Left Preamp Air" 
# IncrementActionCount(ErrorString,ActiontoChange)

# Working
# addNewAction(ErrorString,NewActionString)
# deleteError(ErrorString)

# Testing
# deleteAction(ErrorString,ActiontoDelete)
